# Sentence Similarity Measures I: Count/Tf-Idf Family

## 0. Contents

* I. Corpora
* II. Baseline
    * Word Overlap (Naive Overlap; Word-Discriminativeness-Based Overlap)
    * TF-IDF Overlap
* III. Relative-Frequency Measure
* IV. Probabilistic Model
* V. Evaluation

## I. Corpora

### A. MSR Paraphrase Corpus

##### Load

In [17]:
import pandas as pd

In [1]:
path = "/Users/jacobsw/Desktop/WORK/OJO_CODE/SENTENCE_SIMILARITIES/CORPORA/paraphrase/msr_paraphrase_data.txt"

In [14]:
df = pd.read_csv(path, delimiter='\t')
df.head()

,﻿Sentence ID,String,Author,URL,Agency,Date,Web Date
0,702876,"Amrozi accused his brother, whom he called ""th...",Darren Goodsir,www.theage.com.au,*,June 5 2003,2003/06/04
1,702977,"Referring to him as only ""the witness"", Amrozi...",Darren Goodsir,www.smh.com.au,Sydney Morning Herald,June 5 2003,2003/06/04
2,2108705,Yucaipa owned Dominick's before selling the ch...,MICHAEL GIBBS,www.nwherald.com,*,*,2003/08/23
3,2108831,Yucaipa bought Dominick's in 1995 for $693 mil...,ALEX VEIGA,www.miami.com,*,*,2003/08/23
4,1330381,They had published an advertisement on the Int...,Philip Pangalos,www.alertnet.org,*,*,2003/06/25


In [16]:
len(df['String'])

10594

In [19]:
data = list(df['String'])

In [21]:
for i in xrange(5):
    print data[i]
    print

Amrozi accused his brother, whom he called "the witness", of deliberately distorting his evidence.

Referring to him as only "the witness", Amrozi accused his brother of deliberately distorting his evidence.

Yucaipa owned Dominick's before selling the chain to Safeway in 1998 for $2.5 billion.

Yucaipa bought Dominick's in 1995 for $693 million and sold it to Safeway for $1.8 billion in 1998.

They had published an advertisement on the Internet on June 10, offering the cargo for sale, he added.



##### To Lemmas

In [18]:
from spacy.en import English

In [22]:
parser = English()

In [28]:
def parse_msr():
    
    parsed_sents = [parser(unicode(sent.decode('utf8','ignore'))) for sent in data]
    lemma_sents = [[token.lemma_ for token in parsed_sent] 
                   for parsed_sent in parsed_sents]
    
    return lemma_sents

In [30]:
%%time
sents = parse_msr()

CPU times: user 20.2 s, sys: 110 ms, total: 20.3 s
Wall time: 20.4 s


In [32]:
for i in xrange(5):
    print sents[i]
    print

[u'amrozi', u'accuse', u'his', u'brother', u',', u'whom', u'he', u'call', u'"', u'the', u'witness', u'"', u',', u'of', u'deliberately', u'distort', u'his', u'evidence', u'.']

[u'refer', u'to', u'him', u'as', u'only', u'"', u'the', u'witness', u'"', u',', u'amrozi', u'accuse', u'his', u'brother', u'of', u'deliberately', u'distort', u'his', u'evidence', u'.']

[u'yucaipa', u'own', u'dominick', u"'s", u'before', u'sell', u'the', u'chain', u'to', u'safeway', u'in', u'1998', u'for', u'$', u'2.5', u'billion', u'.']

[u'yucaipa', u'buy', u'dominick', u"'s", u'in', u'1995', u'for', u'$', u'693', u'million', u'and', u'sell', u'it', u'to', u'safeway', u'for', u'$', u'1.8', u'billion', u'in', u'1998', u'.']

[u'they', u'have', u'publish', u'an', u'advertisement', u'on', u'the', u'internet', u'on', u'june', u'10', u',', u'offer', u'the', u'cargo', u'for', u'sale', u',', u'he', u'add', u'.']



## II. Baselines

**NB**:

* $q, r$: *query sentence* and *candidate sentence* respectively.

### A. Word Overlap

**Math**

* **Variant I**: 
    * Equation: $SIM(q,r) = \frac{|q|\cap|r|}{|q|}$ (cf. Metzler et al. (2005):3)

* **Variant II**: 
    * Equation: $SIM(q,r) = \frac{|q|\cap|r|}{|q|}\left(\sum_{w\in q\cap r}log\frac{N}{df_w}\right)$ (cf. ibid.)
    * Notation: $N$: total number of documents; $df_w$: document frequency of word $w$.
    * Idea: "... high IDF terms are typically stronger indicators of shared heritage between two sentences than are low IDF terms" (cf. ibid.)


In [41]:
from __future__ import division
import numpy as np

In [38]:
def word_overlap1(q, r):
    
    return len(set(q).intersection(set(r))) / len(q)

In [39]:
q = sents[0]
r1 = sents[1] # known to be the paraphrase of q
r2 = sents[2] # known to not be the paraphrase of q

In [48]:
%%time
print word_overlap1(q,r1)
print word_overlap1(q,r2)

0.684210526316
0.105263157895
CPU times: user 106 µs, sys: 36 µs, total: 142 µs
Wall time: 119 µs


In [44]:
log = lambda x: np.log(x) if x>0 else np.log(1e-20)
div = lambda x,y: x/y if y!=0 else 0

In [45]:
df_w = lambda w: sum(1 if w in sent else 0 for sent in sents)

In [46]:
def word_overlap2(q, r):
    
    w_list = list(set(q).union(set(r)))
    N = len(sents)
    
    return word_overlap1(q, r) * sum(log(div(N,df_w(w))) for w in w_list)

In [49]:
%%time
print word_overlap2(q,r1)
print word_overlap2(q,r2)

55.7708837465
14.0312695745
CPU times: user 363 ms, sys: 14.1 ms, total: 377 ms
Wall time: 368 ms


### B. TF-IDF Overlap

**Math**

* **Allan et al. (2003)'s TF-IDF**
    * Equation: $SIM(q,r) = \sum_{w\in q\cap r} log(tf_{w,q} + 1)log(tf_{w,r} + 1)log\left(\frac{N+1}{df_w+0.5}\right)$ (cf. Metzler et al. (2005):3)
    * Notation: $tf_{w,q}, tf_{w,r}$: the number of times term $w$ appears in $q,r$ respectively. the rest are the same as before.
    * Idea: "... the more frequently a word appears in a passage, the more indicative that word is of the topicality of that passage; and that the less frequently a term appears in a collection, the greater its power to discriminate between interesting and uninteresting passages." (cf. ibid.)

In [52]:
tf = lambda w,s: s.count(w)

In [55]:
def tf_idf(q, r):
    
    w_list = list(set(q).union(set(r)))
    N = len(sents)
    
    return sum(log(tf(w,q)+1)*log(tf(w,r)+1)*log(div(N+1,df_w(w)+.5)) 
               for w in w_list)

In [56]:
%%time
print tf_idf(q,r1)
print tf_idf(q,r2)

28.3208219916
0.162544003972
CPU times: user 352 ms, sys: 33.4 ms, total: 385 ms
Wall time: 362 ms


## III. Relative-Frequency Measure

**Math**

* **Hoad & Zobel (2003)'s RF**
    * Equation: $SIM(q,r) = \frac{1}{1+\frac{max(|q|,|r|)}{min(|q|,|r|)}}\sum_{w\in q\cap r}\frac{logN/df_w}{1+|tf_{w,q}-tf_{w,r}|}$ (cf. Metzler et al. (2005):4)
    * Idea: 
        * Discriminator: $log\frac{N}{df_w}$ (i.e. TF-IDF)
        * Penalizer 1: $\frac{max(|q|,|r|)}{min(|q|,|r|)}$, penalizes differences in the overall lengths of the sentences.
        * Penalizer 2: $|tf_{w,q}-tf_{w,r}|$, penalizes inequalities in the relative frequency of a word between the two sentences.

In [57]:
def rf(q, r):
    
    w_list = list(set(q).union(set(r)))
    N = len(sents)
    
    return 1/(1+div(max(len(q),len(r)),min(len(q),len(r)))) * \
           sum(div(log(N/df_w(w)),1+(tf(w,q)-tf(w,r))) for w in w_list)

In [58]:
%%time
print rf(q,r1)
print rf(q,r2)

27.8548071877
14.670872733
CPU times: user 372 ms, sys: 23.6 ms, total: 395 ms
Wall time: 380 ms


## IV. Probabilistic Models

**Math**

* **Metzler et al.'s Statistical Translation Based Similarity**
    * Equation: $SIM(q,r) = \prod_{i=1}^{|q|}\frac{tf_{q_i,r}+\mu\cdot p(q_i|C)}{|r| + \mu}$ (cf. Metzler et al. (2005):4)
    * Notation: $\mu$: the number of translate-to-null nodes in an alignment (cf. IBM Translation Model 1); $C$: background model for term (conditional) frequencies inferred from corpus.
    * Assumptions: 
        * The candidate sentence $r$'s possible alignments to the query sentence $q$ are uniformly distributed.
        * The probability that $r$ translates into a sentence of length $\mathcal{l}$ is uniformly distributed.
        * In the absence of any other evidence, an unalgined word is likely to be present in a sentence with a probability equal to its overall probability in the more generalized background language model. (i.e. $p_t(q_i|r_j) = p(q_i|C)$).
        * Each word translates to itself (i.e. $p_t(q_i|r_j)=1$ if $q_i=r_j$).
    * Hyperparameter:
        * $\mu$: the closer $\mu$ is to $0$, the closer the model is to a *word overlap* type of measure (good at finding exact matches), on the other end of the spectrum, the model will be finding *topically relevant matches*. $\mu = 1$ and $\mu = 2500$ are explored. (cf. ibid.)

In [60]:
from operator import mul

In [74]:
total_w_count = sum(1 for sent in sents for w in sent)
p_qi_C = lambda q_i: div(sum(sent.count(q_i) for sent in sents),total_w_count)  # C = sents, our corpus.
prod = lambda l: reduce(mul, l)

In [196]:
def stat_trans_sim(q, r, mu=1):
    
    return prod(div(tf(q[i],r)+mu*p_qi_C(q[i]),len(r)+mu)
                for i in xrange(len(q)))

In [197]:
%%time
print stat_trans_sim(q,r1)
print stat_trans_sim(q,r2)
print
print stat_trans_sim(q,r1,2500)
print stat_trans_sim(q,r2,2500)

4.92226629921e-34
1.91231846503e-76

2.82427653851e-49
2.53971502954e-55
CPU times: user 566 ms, sys: 32.7 ms, total: 599 ms
Wall time: 575 ms


## V. Evaluation

**Method**

* With the 5297 pairs of paraphrases from MSR Paraphrase Corpus, define a *correct classification* for inputting a sentence $q$ to be retrieving its pairmate as the candidate sentence that has the highest similarity score.
* Due to constraint on computational cost, I sample 100 $q$s and then 10 $r$s for the evaluation. For each $q$, I first compute its similarity to its pairmate (i.e. its true paraphrase), and then iterate through the sampled $r$s. If $q$'s similarity to its pairmate is the max in iterations, I count 1 correct.

In [116]:
import random
import numpy as np
from itertools import product
from collections import defaultdict

In [90]:
s2i = {' '.join(s):i for i,s in enumerate(sents)}

In [118]:
s2s = {i:i+1 for i in xrange(0,len(sents),2)}
s22s1 = {j:i for i,j in s2s.iteritems()}
s2s.update(s22s1) # a dictionary the value idx of a key idx is the key's pairmate.

In [173]:
without = lambda i,size: range(0,i)+range(i+1,size) # return the list of indices that don't include i.

In [190]:
def evaluate(sim, sample_num, base_num): 
    # sample_num: evaluation size.
    # base_num: size of sents to measure sim with.
    
    print "... sample size: %d | base size: %d" % (sample_num,base_num)
    sample_idxs = random.sample(range(len(sents)),sample_num)
    
    accuracies = []
    
    correct = 0
    for i in sample_idxs:
        true_sim = sim(sents[i],sents[s2s[i]]) # sent's similarity to its pairmate.
        base_idxs = random.sample(without(i,len(sents)),base_num)
        fail = False
        for j in base_idxs: 
            if i!=j and sim(sents[i],sents[j])>true_sim:
                fail = True
                break 
        correct += 0 if fail else 1    
    
    print "Accuracy: %.6f%%" % ((correct/sample_num)*100)
            

In [206]:
%%time
evaluate(word_overlap1,sample_num=100,base_num=10)

... sample size: 100 | base size: 10
Accuracy: 61.000000%
CPU times: user 28.5 ms, sys: 1.22 ms, total: 29.7 ms
Wall time: 28.8 ms


In [207]:
%%time
evaluate(word_overlap2,sample_num=100,base_num=10)

... sample size: 100 | base size: 10
Accuracy: 51.000000%
CPU times: user 2min 57s, sys: 1.28 s, total: 2min 58s
Wall time: 2min 58s


In [208]:
%%time
evaluate(tf_idf,sample_num=100,base_num=10)

... sample size: 100 | base size: 10
Accuracy: 52.000000%
CPU times: user 3min 7s, sys: 1.47 s, total: 3min 8s
Wall time: 3min 8s


In [209]:
%%time
evaluate(rf,sample_num=100,base_num=10)

... sample size: 100 | base size: 10
Accuracy: 54.000000%
CPU times: user 2min 50s, sys: 1.31 s, total: 2min 52s
Wall time: 2min 51s


In [210]:
%%time
evaluate(stat_trans_sim,sample_num=100,base_num=10)

... sample size: 100 | base size: 10
Accuracy: 53.000000%
CPU times: user 1min 49s, sys: 801 ms, total: 1min 50s
Wall time: 1min 49s
